In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
data = pd.read_csv('bus1234.csv')
data['day'] = 1
data['day'] = pd.to_datetime(data[['year','month','day']])
data['miles'] =data['miles']/1e6
data['miles_sq'] = data['miles']**2
data['miles_cubed'] = data['miles']**3

In [3]:
test = data.copy()
test = sm.add_constant(test)
test['future_replace'] = data.groupby('id')['replace'].shift(-1)
test['future_miles'] = data.groupby('id')['miles'].shift(-1)
test['past_miles'] =  data.groupby('id')['miles'].shift(1)
test['future_miles_sq'] = test['future_miles']*test['future_miles']
test = test.dropna()

test = test.sort_values(['id','day'])

/home/erichschulman/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [4]:
logit0 = sm.Logit( test['replace'], test[['const','miles']] ).fit()
print(logit0.summary())

Optimization terminated successfully.
         Current function value: 0.037755
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                replace   No. Observations:                 8052
Model:                          Logit   Df Residuals:                     8050
Method:                           MLE   Df Model:                            1
Date:                Thu, 07 Nov 2019   Pseudo R-squ.:                  0.1406
Time:                        14:31:54   Log-Likelihood:                -304.01
converged:                       True   LL-Null:                       -353.74
                                        LLR p-value:                 2.000e-23
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.4388      0.384    -19.349      0.000      -8.192      -6.685
miles         14.4029      1

In [5]:
logit1 = sm.Logit( test['replace'], test[['const','miles','miles_sq']] ).fit()
print(logit1.summary())

Optimization terminated successfully.
         Current function value: 0.036971
         Iterations 12
                           Logit Regression Results                           
Dep. Variable:                replace   No. Observations:                 8052
Model:                          Logit   Df Residuals:                     8049
Method:                           MLE   Df Model:                            2
Date:                Thu, 07 Nov 2019   Pseudo R-squ.:                  0.1584
Time:                        14:31:54   Log-Likelihood:                -297.69
converged:                       True   LL-Null:                       -353.74
                                        LLR p-value:                 4.557e-25
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -10.5703      1.269     -8.327      0.000     -13.058      -8.082
miles         46.7290     11

In [6]:
#add a trend
test['trend'] = np.zeros(test.shape[0])
values = test.groupby(['id']).count()['const']
for ids in test['id'].unique():
    trend = np.arange(values[ids])
    test.loc[test['id']==ids,'trend'] = trend

predictors = np.array( [test['trend']*test['replace'], test['trend']*(1-test['replace'])] )
predictors = predictors.transpose()
miles_ols = sm.OLS(test['miles'],test['trend']).fit()
print(miles_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                  miles   R-squared:                       0.673
Model:                            OLS   Adj. R-squared:                  0.673
Method:                 Least Squares   F-statistic:                 1.656e+04
Date:                Thu, 07 Nov 2019   Prob (F-statistic):               0.00
Time:                        14:31:55   Log-Likelihood:                 8543.3
No. Observations:                8052   AIC:                        -1.708e+04
Df Residuals:                    8051   BIC:                        -1.708e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
trend          0.0022   1.72e-05    128.685      0.0

In [13]:
#note we have normalized V0 = 0, this works because of the renewal process?
V1 = -(1-test['replace'])*(logit1.params*test[['const','miles','miles_sq']]).sum(axis=1)

#roughly predict v next period
predicted_miles = (miles_ols.params['trend'] + test['miles'])*(1-test['replace'])
predicted_miles = np.array([test['const'], predicted_miles, predicted_miles*predicted_miles]).transpose()
predicted_miles = pd.DataFrame(predicted_miles , columns =['const','miles','miles_sq'], index=test.index  )

EV1 = -(logit1.params*predicted_miles).sum(axis=1)
choice_pr = np.exp(EV1)/(1+np.exp(EV1))

U =  V1 - .99*EV1

In [14]:
exog = np.array([ test['const'], test['replace'], test['miles']*(1-test['replace'])])
sm.OLS(U, exog.transpose()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.068e+08
Date:                Thu, 07 Nov 2019   Prob (F-statistic):               0.00
Time:                        14:35:09   Log-Likelihood:                 30324.
No. Observations:                8052   AIC:                        -6.064e+04
Df Residuals:                    8049   BIC:                        -6.062e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1980      0.000   1828.125      0.000       0.198       0.198
x1           -10.6626      0.001  -1.46e+04      0.000     -10.664     -10.661
x2            -0.5762      0.001   -773.919      0.000      -0.578      -0.575
==============================================================================
Omnibus:                     2429.413   Durbin-Watson:                   0.090
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7976.497
Skew:                           1.528   Prob(JB):                         0.00
Kurtosis:                       6.800   Cond. No.                         12.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""